# Setup
1. If you are scripting within an enterprise, replace the base URL with your enterprise's URL. For example, https://ptc.onshape.com if within the PTC enterprise. Note the omitted / on the end!
1. Create a file colabkeys.py in a text editor on your computer.
1. Get an access and secret key pair from https://dev-portal.onshape.com/.
1. Format the file to match this example, but with your own access and secret keys\*:
```
access = 'Z97asb1257FkEwzp3EmCpa'
secret = 'nHwN3q7asdvg4aryFC9rxYyo9U4o415WokYa8VOQ9YfUh4Zx'
```
1. Upload this file to the root directory of your Google Drive's My Drive section.
1. Ensure `colab_keys_path` in the section below is `colabkeys.py` for this location and file name. If your file is named differently or in a different location in your Google Drive, modify the `colab_keys_path` field.
1. Run the cell. If you get warnings about the Drive already being mounted or a client already existing, these are expected if you have previously run this setup script in this session.

\* Note: These are fake keys for demonstration purposes.

In [ ]:
#@title **Client parameters - Run Each Time You Open Colab**

#@markdown Onshape base URL. Change this if in an enterprise (e.g. https://ptc.onshape.com)
base = 'https://liveworx20.onshape.com' #@param {type: "string"}

#@markdown Onshape API key python file path/name relative to My Drive root
#@markdown directory in Google Drive.

colab_keys_path = 'colabkeys.py' #@param {type: "string"}
colab_keys_path = '/content/drive/MyDrive/' + colab_keys_path

# install onshape python client
import sys, os
print('Installing onshape python client...')
with open(os.devnull, 'w') as devnull:
    old_stdout = sys.stdout
    sys.stdout = devnull
    !pip install onshape-client
    sys.stdout = old_stdout

# import onshape python client and json library
from onshape_client.client import Client
import json

# mount Google Drive for API key and data access
from google.colab import drive
print('Mounting Google Drive...', end='\n\n')
drive.mount('/content/drive')

# load API keys from file colabkeys.py in root directory of my drive
print()
if os.path.exists(colab_keys_path): 
    print(colab_keys_path + ' exists in My Drive; loading keys...')
    execfile(colab_keys_path)
else:
    print(colab_keys_path + ' does not exist in My Drive root directory.')
print()

# create onshape client from base URL and keys
print('Constructing Onshape client from base URL ', base, ' and key file ', colab_keys_path, '...', sep='', end='\n\n')
client = Client(configuration={"base_url": base,
                               "access_key": access,
                               "secret_key": secret})
# a delay is required since the client constructor sends out its ready message asynchronously
import time
time.sleep(0.5)
print('\nSetup done.')


Installing onshape python client...
Mounting Google Drive...

Mounted at /content/drive

/content/drive/MyDrive/colabkeys.py exists in My Drive; loading keys...

Constructing Onshape client from base URL https://liveworx20.onshape.com and key file /content/drive/MyDrive/colabkeys.py...


Setup done.


# Your Code

**Make sure you run the setup cell found above at least once before you run your own code for every Colab session.** Otherwise, you will run into authentication and missing dependency errors.

You can add cells by hovering with your mouse near the center of the gap between cells (or at the very bottom of the last cell) and selecting either Code or Text for the type of the new cell.

# Validate Balance

The balance is ok if the centerpoint of the boundingbox of the assembly is close to the center of gravity. 

Close means that the **distance is < 0.05m**.


In [ ]:
#@title Calculated Distance { vertical-output: true }
import json
import math
from IPython.display import display, Markdown

expectedMaxDistance = 0.05

# get boundigbox centerpoint first
fixed_url = '/api/v4/assemblies/d/did/w/wid/e/eid/boundingboxes'

# https://liveworx20.onshape.com/api/v4/assemblies/d/62133c27fba736a62fced394/w/a68f6e8a29cc1bf010fa71d5/e/0124d5a6da75c7ab8de75c57/boundingboxes?includeHidden=false&includeSketches=false
did = '62133c27fba736a62fced394'
wid = 'a68f6e8a29cc1bf010fa71d5'
eid = '0124d5a6da75c7ab8de75c57'

method = 'GET'

params = {
    "includeHidden": False,
    "includeSketches": False
}

headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
           'Content-Type': 'application/json'}

fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body={})
bbox = json.loads(response.data)
centerpoint = [
    (bbox["lowX"]+bbox["highX"])/2, 
    (bbox["lowY"]+bbox["highY"])/2, 
    (bbox["lowZ"]+bbox["highZ"])/2
]

# get center of gravity next
fixed_url = '/api/v4/assemblies/d/did/w/wid/e/eid/massproperties'
params = {}
fixed_url = fixed_url.replace('did', did)
fixed_url = fixed_url.replace('wid', wid)
fixed_url = fixed_url.replace('eid', eid)

response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body={})
massprops = json.loads(response.data)

pointDistance = math.sqrt( (centerpoint[0]-massprops["centroid"][0])**2 + 
                           (centerpoint[1]-massprops["centroid"][1])**2 + 
                           (centerpoint[2]-massprops["centroid"][2])**2
                          )
testPassed = pointDistance < expectedMaxDistance




# rendering of results from here on
display(Markdown("### **Overall Mass of the assembly**: {:.2f} kg ###".format(massprops["mass"][0])))
if(testPassed):
   result = "Test Passed"
   image = "https://pbs.twimg.com/profile_images/1398006785298944000/pH1w1bvv_400x400.png"
else:
   result = "Test Failed"
   image = "https://backgroundlabs.com/files/failed-background-2940.png"

display(Markdown("### {}".format(result)))
display(Markdown("![figure]({})".format(image)))
display(Markdown("**BBox Centerpoint to Center of Gravity Distance:** {:.4f}".format(pointDistance)))



### **Overall Mass of the assembly**: 0.75 kg ###

### Test Passed

![figure](https://pbs.twimg.com/profile_images/1398006785298944000/pH1w1bvv_400x400.png)

**BBox Centerpoint to Center of Gravity Distance:** 0.0071

# Unit Test for Onshape 

In [ ]:
import unittest
import json
import math

maxWeight = 15
maxBBocCenterCoGDistance = 0.05

did = '62133c27fba736a62fced394'
wid = 'a68f6e8a29cc1bf010fa71d5'
eid = '0124d5a6da75c7ab8de75c57'

class TestOnshapeDesign(unittest.TestCase):
    def setUp( self ):
        # get boundigbox centerpoint first
        fixed_url = '/api/v4/assemblies/d/did/w/wid/e/eid/boundingboxes'
        method = 'GET'
        params = {
            "includeHidden": False,
            "includeSketches": False
        }
        headers = {'Accept': 'application/vnd.onshape.v1+json; charset=UTF-8;qs=0.1',
                  'Content-Type': 'application/json'}

        fixed_url = fixed_url.replace('did', did)
        fixed_url = fixed_url.replace('wid', wid)
        fixed_url = fixed_url.replace('eid', eid)
        response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body={})
        global bbox
        bbox = json.loads(response.data)
        # get center of gravity next
        fixed_url = '/api/v4/assemblies/d/did/w/wid/e/eid/massproperties'
        params = {}
        fixed_url = fixed_url.replace('did', did)
        fixed_url = fixed_url.replace('wid', wid)
        fixed_url = fixed_url.replace('eid', eid)

        response = client.api_client.request(method, url=base + fixed_url, query_params=params, headers=headers, body={})
        global massprops
        massprops = json.loads(response.data)


    def test_weight(self):
        self.assertTrue( massprops["mass"][0]<maxWeight, "actual weight {:.2f} kg is not lower than max weight {:.2f} kg".format(massprops["mass"][0], maxWeight) )

    def test_balance(self):
        centerpoint = [
            (bbox["lowX"]+bbox["highX"])/2, 
            (bbox["lowY"]+bbox["highY"])/2, 
            (bbox["lowZ"]+bbox["highZ"])/2
        ]
        pointDistance = math.sqrt( (centerpoint[0]-massprops["centroid"][0])**2 + 
                           (centerpoint[1]-massprops["centroid"][1])**2 + 
                           (centerpoint[2]-massprops["centroid"][2])**2
                          )
        self.assertTrue( pointDistance < maxBBocCenterCoGDistance , "The distance of the bounding box center and the center of gravity is {:.4f} m which is larger than the allowed maximum {:.4f} m".format(pointDistance, maxBBocCenterCoGDistance) )


# Test Runner

In [ ]:
import unittest

if __name__ == "__main__":
   #unittest.main()
   unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 37.446s

OK
